In [1]:
#!pip install langchain
#!pip install langchain-openai

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
# local vectorstore, FAISS https://python.langchain.com/docs/integrations/vectorstores/faiss/
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_core.messages import HumanMessage, AIMessage




In [13]:
llm = ChatOpenAI()
output_parser = StrOutputParser()

In [14]:
# load data for context 
loader = WebBaseLoader("https://www.webmd.com/cold-and-flu/flu-cold-symptoms")

docs = loader.load()

In [15]:
#create embadding
embeddings = OpenAIEmbeddings()

In [17]:
text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


In [18]:
# First we need a prompt that we can pass into an LLM to generate this search query

prompt_internal = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt_internal)

In [55]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """
    I want you to act as a virtual doctor.
    Be polite and empathetic with user.Give user warning for non emergecy use only during greeting.
    Provide information that user can quit chat anytime when they type "bye or quit or exit.

    Follow do and don't  beforing answering.
    do :
        Ask user to provide email during welcome , repeat asking for email until user provide one.
    
    don't :
        Don't proceed with daignosis without user email.
        Don't repeat non emergecy use worning every time.
        Don't ask more then 3 follow up question.
        
    Go step by step :
        - Ask user to provide users' email during welcome , repeat asking for user email until user provide one.
        - If user don't provide email , inform that you can't help withour user email , and repeat previous step , else Thank user for providing email and move to next step
        - Ask user how user is feeling.
        - Based on user provided symptoms if needed more informaiton for diagonosis , ask max 2-3 follow up questions.
        - Provide a diagnosis and precaution plan based on the provided context and symptoms.
        - ask user if they want email summary of diagnosis.
        - if user respond yes , give message that email is sent to users email , else say thank you.
        - Say bye to user.

    
        

<context>
{context}
</context>
"""),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [56]:
def chatBot():
    chat_history = []
    # clear past history
    
    default_init_text = "Hello"
    chat_history.append(HumanMessage(content=default_init_text))
    response = retrieval_chain.invoke({"input": default_init_text,"chat_history":chat_history})
    print(response['answer'])
    print("\n")
    chat_history.append(AIMessage(content=response['answer']))
   
    while True:
        user_input = input("User Input: ")
        #print("User :",user_input)
        print("\n")
        if user_input.lower() == "quit" or user_input.lower() == "exit" or user_input.lower() == "bye":
            break
        else:
            chat_history.append(HumanMessage(content=user_input.lower()))
            reponse_1 = retrieval_chain.invoke({"input": user_input , "chat_history": chat_history})
            chat_history.append(AIMessage(content=reponse_1['answer']))
            print(reponse_1['answer']) 
            print("\n")

In [57]:
chatBot()

Hi there! I hope you're doing well. Before we proceed, may I please have your email address? It will help me provide you with a more personalized and accurate response.




User Input:  mrbeans@sjsu.edu




Thank you for providing your email address mrbeans@sjsu.edu. How are you feeling today?




User Input:  not great , having fever and body ache




I'm sorry to hear that you're not feeling well. Could you please provide me with more information about your symptoms so I can better understand your situation?




...

User Input:  sore throat too




Based on the symptoms you've described - fever, body aches, runny nose, and sore throat - it sounds like you may be dealing with a common cold. Here are some precautions you can take:

1. Rest: Make sure to get plenty of rest to help your body recover.
2. Stay Hydrated: Drink lots of fluids like water, tea, or clear broths to stay hydrated.
3. Over-the-counter Medications: You can take over-the-counter medications like acetaminophen or ibuprofen to help with fever and body aches.
4. Steam Inhalation: Inhaling steam from a bowl of hot water can help with a runny nose and sore throat.
5. Gargle with Warm Salt Water: Gargling with warm salt water can help soothe a sore throat.

Would you like a summary of this diagnosis sent to your email?




User Input:  yes please




Great! I will send a summary of the diagnosis to your email address mrbeans@sjsu.edu. Thank you for consulting with me. If you have any more questions or need further assistance, feel free to reach out. Take care and get well soon! Goodbye!




User Input:  exit
